In [1]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
file_name = 'feature_extractions_data.csv'

# Get the current working directory
current_dir = '/Users/tony/Desktop'
#file_path_feature_data=current_dir+'\dataset' (这是原代码，下面是更新后的代码：
#使用os.path.join来构建file_path_feature_data，这样可以确保路径在不同的操作系统上都能正确工作)
file_path_feature_data = os.path.join(current_dir, 'CoffeeBeansT')
print(current_dir)
# Combine the current working directory and the file name to get the full file path
file_path = os.path.join(file_path_feature_data, file_name)
print(file_path)



# Get the current working directory
current_dir = '/Users/tony/Desktop'
print(current_dir)
print(file_path_feature_data)
# List all files in the current directory
files_in_current_dir = os.listdir(current_dir)

# Check if the file is in the current directory
#if file_name in files_in_current_dir:
#    print("File found.")
#     trad_df = pd.read_csv(file_path, index_col = False)
#else:
#    print("File not found in the current directory.")
    
    
#z直接使用‘os.path.exists(file_path)'会更加直接和高效
if os.path.exists(file_path):
    print("File Found")
else:
    print("File not Found")

/Users/tony/Desktop
/Users/tony/Desktop/CoffeeBeansT/feature_extractions_data.csv
/Users/tony/Desktop
/Users/tony/Desktop/CoffeeBeansT
File Found


In [ ]:
#使用‘panda’的‘read_csv'函数读取'file_path'路径的CSV文件，结果存储在DataFrame 'trad_df'中。参数'index_col = False'表示不将任何列用作行索引
try:
    trad_df = pd.read_csv(file_path, index_col = False) 
    # trad_df.drop("Unnamed: 0",axis=1,inplace=True)?
    # trad_df.drop(columns=["label", "filename"], inplace=False)
    # trad_df["label"]
except FileNotFoundError:
    print(f"error: file {file_path} not found. 请检查文件路径是否正确")
    #可以选择退出程序或者继续执行其他代码
    #exit() 或 return
except PermissionError:
    print(f"error: 没有权限读取文件{file_path}. 请检查文件权限")
    # exit() or return
except pd.errors.EmptyDataError:
    print(f"error: file {file_path} is empty")
    #exit() or return
except Exception as e:
    print(f"读取文件{file_path}时发生未知错误: {e}")
    #exit() or return
trad_df

In [ ]:
    # drop string table
    #这行代码使用'drop'方法从'trad_df' Dataframe中移除'"path"'列。参数'axis=1'表示操作的是列而不是行。
trad_df = trad_df.drop("path", axis=1)
trad_df = trad_df.drop("filename", axis=1)

#KNN method 
from sklearn.impute import KNNImputer
knn_imputer = KNNImputer(n_neighbors=5, weights="uniform")
trad_df_imputed = knn_imputer.fit_transform(trad_df)
trad_df_imputed = pd.DataFrame(trad_df_imputed, columns=trad_df.columns)

trad_df

In [ ]:
#Average padding
#Note: This only applies to numeric columns

#trad_df.fillna(trad_df.mean(), inplace=True)


#Median fill:
#trad_df.fillna(trad_df.median(), inplace=True)


#Linear interpolation:
#这适用于连续数值数据，假设数据是线性分布的
#This applies to continuous numerical data, assuming that the data is linearly distributed

#trad_df.interpolate(method='linear', inplace=True)

In [ ]:
#Plurality Filling:
#这适用于数值和分类数据。 This applies to numerical and categorical data

for column in trad_df.columns:
    trad_df[column].fillna(trad_df[column].mode()[0], inplace=True)

In [ ]:
#unique_label = np.unique(trad_df['label'])
#int_label = []
#for l in trad_df['label']:
#  result = [x == l for x in unique_label]
#  i = np.argmax(result)
#  int_label.append(i)

#trad_df['label'] = int_label


trad_df['label'], _ = pd.factorize(trad_df['label'])

In [ ]:
#View the data type of each column in the 'trad_df' DataFrame.
trad_df.dtypes

#此属性返回一个Series，显示'trad_df'中每列的数据类型。对于Pandas DataFrame来说，常见的数据类型包括'int64'(整数)、'float64'(浮点数)、'object'(通常是字符串)等。

In [ ]:
#The 'reset_index' method is used to reset the index of the 'trad_df' DataFrame, and by setting the 'drop=True' parameter, it does not add the old index as a column of the DataFrame when resetting the index.
trad_df = trad_df.reset_index(drop=True)

#将'trad_df'的索引重置为默认的整数索引（从0开始），并且通过指定的'drop=True'来丢弃旧的索引。
#如果不设置'drop=True'，旧的索引会被保留下来作为一个新的列。

In [ ]:
#Checks the number of missing values per column in the 'trad_df' DataFrame.
trad_df.isnull().sum()

#'trad_df.isnull()' 这个方法返回一个与'trad_df'形状相同的布尔类型DataFrame，其中的值表示相应位置的数据是否为缺失值（'True'表示缺失值，'False'表示非缺失值）
#'.sum()' 当在'isnull()‘方法的结果上调用'sum()'方法时，它会对每列的布尔值('True'计为0)求和，从而给出每列缺失值的总数

In [ ]:
#将无限值替换为NaN，然后删除含有NaN值的行以清洗数据集：
trad_df.replace([np.inf, -np.inf], np.nan, inplace=True)
trad_df.dropna(inplace=True)

In [ ]:
trad_df

In [ ]:
# split data for training purpose
from sklearn.model_selection import train_test_split
X = trad_df.drop("label",axis=1)
y = trad_df["label"]
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2)
X_test, X_validate, y_test, y_validate = train_test_split(X_val,y_val,test_size=0.5)


In [ ]:
from sklearn import tree
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from math import sqrt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.neighbors import (NeighborhoodComponentsAnalysis, KNeighborsClassifier)
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.callbacks import EarlyStopping

from sklearn.datasets import load_iris
import joblib



from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# modelling

def get_model(index = -1): 
  models = [
  {
      "name" : "RandomForestClassifier",
      "model" : RandomForestClassifier()
  },
  {
      "name" : " KNeighborsClassifier",
      "model" :  KNeighborsClassifier()
  },
  {
      "name" : "DecisionTreeClassifier",
      "model" : tree.DecisionTreeClassifier()
  },
    {
      "name" : "GradientBoostingClassifier",
      "model" : GradientBoostingClassifier()
  }]

  if index == -1:
    return len(models)

  return models[index]

In [ ]:
def train_model(model):                                #定义了一个名为'train_model'的函数，它接收了一个‘model’字典作为参数，这个字典预期包含模型的名称和实例。
  print("Traning by {} ...".format(model["name"]))

  try:
    m = model["model"]
  
    m.fit(X_train,y_train)                               #使用训练数据对模型进行拟合

    
  
    score = m.score(X_validate,y_validate)               #在验证集上评估模型性能
    # Make predictions on the test data
    y_pred = m.predict(X_validate)                       #对验证集进行预测

  # Calculate evaluation metrics
    accuracy = accuracy_score(y_validate, y_pred)
    precision = precision_score(y_validate, y_pred, average='weighted')
    recall = recall_score(y_validate, y_pred, average='weighted')
    f1 = f1_score(y_validate, y_pred, average='weighted')
  #   losses = mean_squared_error(y_validate, y_pred,squared=False)
    losses = mean_squared_error(y_validate, y_pred)
    loss = f"{losses:.4f}"
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")
    print(f"Loss: {loss}")
    print(f"Loss (MSE): {losses:.4f}")

    return m,score,precision,recall,f1,loss
  


  
  except ValueError as e:
    print(f"模型训练失败，数据问题: {e}")
  except KeyError as e:
    print(f"模型训练失败，缺少必要的键: {e}")
  except Exception as e:
    print(f"模型训练时发生未知错误: {e}")

In [ ]:
## main execution

models = []
scores = []
precisions = []
recalls = []
f1s = []
loss = []
work_model= []

for i in range(get_model()):
  model_wrapper = get_model(i)
  model = model_wrapper["model"]
  model_name = model_wrapper["name"]

  m,score,precision,recall,f1,losses = train_model(model_wrapper)
#   train_model_deep_learning(model_wrapper)
  models.append(model_name)
  scores.append(score)
  precisions.append(precision)
  recalls.append(recall)
  f1s.append(f1)
  loss.append(losses)
  work_model.append(m)

In [ ]:
#import json
#def generate_score_df(models = models, scores = scores, precision = precisions, recall = recalls, f1 = f1s,loss = loss ):
#  df_scores = pd.DataFrame(np.array([models,scores,precision,recall,f1,loss]).T,columns=["model_name","Accuracy","precision","recall","f1","loss"])


#  return df_scores

In [ ]:
import json
def generate_score_df(models=models, scores=scores, precision=precisions, recall=recalls, f1=f1s, loss=loss):
    try:
        # 检查所有列表长度是否一致
        lengths = [len(models), len(scores), len(precision), len(recall), len(f1), len(loss)]
        if len(set(lengths)) > 1:
            raise ValueError("The length of the input list is not consistent, please check the input data")
        df_scores = pd.DataFrame(np.array([models, scores, precision, recall, f1, loss]).T, columns=["model_name", "Accuracy", "precision", "recall", "f1", "loss"])
    except ValueError as e:
        print(f"Failed to generate scoring DataFrame:{e}")
        return None
    except Exception as e:
        print(f"An unknown error occured while generating the scoring DataFrame: {e}")
        return None
    return df_scores


In [ ]:
generate_score_df()